<header>
   <p  style='font-size:36px;font-family:Arial;color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Interactive NPath Widget - generate Sankey and Icicle Charts from Event Logs
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:18px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>This Notebook demonstrates a No-Code ipywidget that allows business users to leverage the power of NPath path analysis.</p>
<p style = 'font-size:16px;font-family:Arial'>Execute the next cell to watch a video that is an example of using the widget.</p>

In [ ]:
from IPython.display import HTML
HTML("""
<video width="700" height="700" controls> <source src="./images/demo.gif.mp4" type="video/mp4"> </video>
""")

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>1. Connect to Vantage.</b></p>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).</p>

In [ ]:
%%capture
# # '%%capture' suppresses the display of installation steps of the following packages
!pip install --upgrade tdnpathviz>=0.1.3.1

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial'><b>Note: </b><i>The kernel must be restarted to bring the installed libraries into memory. Please select <b>Kernel</b> from the menu and then select <b>Restart Kernel...</b></i></p>
</div>

In [ ]:
from teradataml import create_context, execute_sql, db_list_tables, remove_context, db_drop_view
from tdnpathviz import npath_widget

<p style = 'font-size:16px;font-family:Arial'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_NPath_Visualizer_Widget.ipynb;' UPDATE FOR SESSION; ''')

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>2. Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>For this demo we will use data that has been originally prepared for a set of other demos. Like this, we can switch between different types of data and show the versatility of the NPath widget.</p>   


In [ ]:
# Takes about 1 minute
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_DigitalEvents_cloud');"
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_KneeReplacement_cloud');"
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_TrainDelay_cloud');"
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_MultiTouchAttribution_cloud');"

<p style = 'font-size:16px;font-family:Arial'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>3. Create Database with View pointing at the events tables.</b></p>

<p style = 'font-size:16px;font-family:Arial'>For this demo, we want to have all the data for the exploration in the same database. The easiest way to do this by creating a database that just contains views pointing at the tables. This is necessary because the <code>tdnpathviz.npath_widget()</code> function requires a database name as input.</p>



In [ ]:
execute_sql("CREATE DATABASE npath_input FROM demo_user AS PERMANENT = 100")
execute_sql("REPLACE VIEW npath_input.Digital_Retail_Events  AS SEL * FROM DEMO_DigitalEvents.Digital_Retail_Events")
execute_sql("REPLACE VIEW npath_input.Knee_Replacement  AS SEL * FROM DEMO_KneeReplacement.Knee_Replacement")
execute_sql("REPLACE VIEW npath_input.Train_Dataset  AS SEL * FROM DEMO_TrainDelay.Train_Dataset")
execute_sql("REPLACE VIEW npath_input.Attribution_Data  AS SEL * FROM DEMO_MultiTouchAttribution.Attribution_Data")

<p style = 'font-size:16px;font-family:Arial'>Now let's check if we were successful.</p>

In [ ]:
db_list_tables(schema_name = "npath_input")

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>4. NPath Widget</b></p>
<p style = 'font-size:16px;font-family:Arial'>We will start by trying out the widget. Be sure to test all different tables with different settings and different charts. You can also save the charts in files.</p>

In [ ]:
npath_widget(database="npath_input")

<br>
<br>
<br>
<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>5. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>We need to clean up our work tables to prevent errors next time.</p>

In [ ]:
views = ['Digital_Retail_Events', 'Knee_Replacement', 'Train_Dataset','Attribution_Data']

# Loop through the list of views and execute the drop view command for each view
for view in views:
    try:
        db_drop_view(view_name = view, schema_name = 'npath_input') 
    except:
        pass

<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>We will use the following code to clean up tables and databases created for this demonstration.</p>

In [ ]:
execute_sql("DROP DATABASE npath_input")

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_DigitalEvents');" 
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_KneeReplacement');" 
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_TrainDelay');" 
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_MultiTouchAttribution');" 

In [ ]:
remove_context()

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>Required Materials</b>
<p style = 'font-size:16px;font-family:Arial'>Let’s look at the elements we have available for reference for this notebook:</p>
<p style = 'font-size:16px;font-family:Arial'>Check more details regarding the tdnpathviz package <a href='https://pypi.org/project/tdnpathviz/'>here.</a></p>

<b style = 'font-size:18px;font-family:Arial'>Related Resources:</b>
<li style = 'font-size:16px;font-family:Arial'><a href = 'https://docs.teradata.com/search/all?query=NPath&content-lang=en-US'>NPath documentation</a> </li>




<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved.
        </div>
    </div>
</footer>